<h1>Getting Started</h1>
1. signup for Rasgo (https://app.rasgoml.com/account/register) and get your API key from the top right corner in the UI<br>
2. install pyrasgo (next cell down)<br>
3. check out the docs: https://docs.rasgoml.com/rasgo-docs/transforms/overview<br>
4. follow along with this tutorial

In [2]:
!pip install pyrasgo --upgrade
import pyrasgo
import pandas as pd

In [45]:
#use your rasgo API key to connect
api_key = ''
rasgo = pyrasgo.connect(api_key)

In [28]:
# Get all available transforms
# Best place to see more info is https://docs.rasgoml.com/rasgo-docs/transforms/overview
transforms = rasgo.get.transforms()
for t in transforms:
    args = []
    [args.append(a.name) for a in t.arguments]
    print(t.id, t.name, '({})'.format(', '.join(args)))

230 lag (columns, amounts, partition, order_by)
227 levenshtein (columns1, column2)
214 datetrunc (date_columns, date_part)
234 filter (filter_statements)
217 train_test_split (order_by, train_percent)
242 simple_join (join_type, join_table, join_columns)
228 moving_avg (input_columns, window_sizes, order_by, partition)
238 aggregate (group_items, aggregations)
218 standard_scaler (columns_to_scale, averages, standarddevs)
226 datediff (date_part, date_val_1, date_val_2)
240 union (source, union_all)
221 cast (casts)
231 if_then (conditions, default, alias)
229 rename (renames)
224 min_max_scaler (columns_to_scale, minimums, maximums)
213 one_hot_encode (column)
237 join (join_table, join_type, join_columns)
225 substring (target_col, start_pos, end_pos)
216 concat (concat_list)
241 datespine (date_col, interval_type, start_timestamp, end_timestamp)
215 datepart (dates)
239 dedupe (natural_key, order_method, order_col)
232 drop_columns (include_cols, exclude_cols)
233 order (col_list, 

In [30]:
# get some rasgo data sources to operate on
internet_sales = rasgo.get.dataset(74)
customer = rasgo.get.dataset(55)
product = rasgo.get.dataset(75)

<h2>Example 1: Forecast Direct Sales</h2>

In [31]:
# preview a dataset
internet_sales.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07


In [18]:
product.preview()

,PRODUCTKEY,PRODUCTALTERNATEKEY,PRODUCTSUBCATEGORYKEY,WEIGHTUNITMEASURECODE,SIZEUNITMEASURECODE,ENGLISHPRODUCTNAME,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,STANDARDCOST,FINISHEDGOODSFLAG,...,CHINESEDESCRIPTION,ARABICDESCRIPTION,HEBREWDESCRIPTION,THAIDESCRIPTION,GERMANDESCRIPTION,JAPANESEDESCRIPTION,TURKISHDESCRIPTION,STARTDATE,ENDDATE,STATUS
0,1,AR-5381,NaN,None,None,Adjustable Race,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
1,2,BA-8327,NaN,None,None,Bearing Ball,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
2,3,BE-2349,NaN,None,None,BB Ball Bearing,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
4,5,BL-2036,NaN,None,None,Blade,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
5,6,CA-5965,NaN,None,None,LL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
6,7,CA-6738,NaN,None,None,ML Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
7,8,CA-7457,NaN,None,None,HL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
8,9,CB-2903,NaN,None,None,Chainring Bolts,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
9,10,CN-6137,NaN,None,None,Chainring Nut,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current


In [32]:
# join internet_sales and product tables
ds1 = internet_sales.join(join_table=product, join_type='LEFT', join_columns={'PRODUCTKEY':'PRODUCTKEY'})

ds1.preview()

,DISCOUNTAMOUNT,FREIGHT,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,CARRIERTRACKINGNUMBER,ORDERDATE,...,WEIGHTUNITMEASURECODE,STYLE,THAIDESCRIPTION,GERMANDESCRIPTION,ENDDATE,STARTDATE,WEIGHT,SIZE,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME
0,0.0,89.4568,21768,2011-01-10,1,19,3578.2700,20110105,None,2010-12-29,...,LB,U,จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับกา...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",2007-12-28,2011-07-01,15.00,62,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62"
1,0.0,84.9998,28389,2011-01-10,1,39,3399.9900,20110105,None,2010-12-29,...,LB,U,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,"""Top-Mountainbike für Profis. Mit innovativem ...",2007-12-28,2011-07-01,21.13,44,"Montaña: 100, plateada, 44","VTT 100 argent, 44"
2,0.0,84.9998,25863,2011-01-10,1,100,3399.9900,20110105,None,2010-12-29,...,LB,U,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,"""Top-Mountainbike für Profis. Mit innovativem ...",2007-12-28,2011-07-01,21.13,44,"Montaña: 100, plateada, 44","VTT 100 argent, 44"
3,0.0,17.4775,14501,2011-01-10,1,100,699.0982,20110105,None,2010-12-29,...,LB,U,จักรยานที่ให้ความคุ้มค่า ด้วยคุณสมบัติพิเศษมาก...,"""Kostengünstiges Rad mit vielen Funktionen uns...",2007-12-28,2011-07-01,20.00,62,"Carretera: 650, negra, 62","Vélo de route 650 noir, 62"
4,0.0,84.9998,11003,2011-01-10,1,6,3399.9900,20110105,None,2010-12-29,...,LB,U,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,"""Top-Mountainbike für Profis. Mit innovativem ...",2007-12-28,2011-07-01,21.13,44,"Montaña: 100, plateada, 44","VTT 100 argent, 44"
5,0.0,89.4568,27645,2011-01-11,1,100,3578.2700,20110106,None,2010-12-30,...,LB,U,จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับกา...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",2007-12-28,2011-07-01,13.77,44,"Carretera: 150, roja, 44","Vélo de route 150 rouge, 44"
6,0.0,89.4568,16624,2011-01-11,1,6,3578.2700,20110106,None,2010-12-30,...,LB,U,จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับกา...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",2007-12-28,2011-07-01,15.00,62,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62"
7,0.0,84.3748,11005,2011-01-11,1,6,3374.9900,20110106,None,2010-12-30,...,LB,U,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,"""Top-Mountainbike für Profis. Mit innovativem ...",2007-12-28,2011-07-01,21.42,48,"Montaña: 100, negra, 48","VTT 100 noir, 48"
8,0.0,84.9998,11011,2011-01-11,1,6,3399.9900,20110106,None,2010-12-30,...,LB,U,สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด...,"""Top-Mountainbike für Profis. Mit innovativem ...",2007-12-28,2011-07-01,20.35,38,"Montaña: 100, plateada, 38","VTT 100 argent, 38"
9,0.0,89.4568,27621,2011-01-12,1,100,3578.2700,20110107,None,2010-12-31,...,LB,U,จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับกา...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",2007-12-28,2011-07-01,14.13,48,"Carretera: 150, roja, 48","Vélo de route 150 rouge, 48"


In [34]:
print(ds1.sql)

CREATE OR REPLACE RASGO_OP_CEB93AB01CEF4C6AB55563AC5D165F4 AS SELECT
  t1.DISCOUNTAMOUNT, 
  t1.FREIGHT, 
  t1.CUSTOMERKEY, 
  t1.DUEDATE, 
  t1.ORDERQUANTITY, 
  t1.CURRENCYKEY, 
  t1.UNITPRICE, 
  t1.SHIPDATEKEY, 
  t1.CARRIERTRACKINGNUMBER, 
  t1.ORDERDATE, 
  t1.SALESTERRITORYKEY, 
  t1.REVISIONNUMBER, 
  t1.CUSTOMERPONUMBER, 
  t1.UNITPRICEDISCOUNTPCT, 
  t1.DUEDATEKEY, 
  t1.SALESORDERLINENUMBER, 
  t1.PROMOTIONKEY, 
  t1.SHIPDATE, 
  t1.TOTALPRODUCTCOST, 
  t1.SALESAMOUNT, 
  t1.TAXAMT, 
  t1.ORDERDATEKEY, 
  t1.EXTENDEDAMOUNT, 
  t1.PRODUCTKEY, 
  t1.SALESORDERNUMBER, 
  t1.PRODUCTSTANDARDCOST, t2.STATUS
    , t2.JAPANESEDESCRIPTION
    , t2.STANDARDCOST
    , t2.SIZEUNITMEASURECODE
    , t2.CLASS
    , t2.DEALERPRICE
    , t2.MODELNAME
    , t2.REORDERPOINT
    , t2.CHINESEDESCRIPTION
    , t2.PRODUCTLINE
    , t2.PRODUCTSUBCATEGORYKEY
    , t2.FINISHEDGOODSFLAG
    , t2.LISTPRICE
    , t2.ARABICDESCRIPTION
    , t2.COLOR
    , t2.DAYSTOMANUFACTURE
    , t2.ENGLISHPRODUCTNAME


In [35]:
# convert date string to date

ds2 = ds1.to_date(dates={'ORDERDATEKEY':'YYYYMMDD'})

ds2.preview()

,DISCOUNTAMOUNT,FREIGHT,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,REVISIONNUMBER,EXTENDEDAMOUNT,...,GERMANDESCRIPTION,CHINESEDESCRIPTION,PRODUCTLINE,PRODUCTSUBCATEGORYKEY,FINISHEDGOODSFLAG,COLOR,DAYSTOMANUFACTURE,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,ORDERDATEKEY_DATE
0,0.0,89.4568,21768,2011-01-10,1,19,3578.2700,20110105,1,3578.2700,...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛...,R,2,1,Red,4,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62",2010-12-29
1,0.0,84.9998,28389,2011-01-10,1,39,3399.9900,20110105,1,3399.9900,...,"""Top-Mountainbike für Profis. Mit innovativem ...",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,M,1,1,Silver,4,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29
2,0.0,84.9998,25863,2011-01-10,1,100,3399.9900,20110105,1,3399.9900,...,"""Top-Mountainbike für Profis. Mit innovativem ...",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,M,1,1,Silver,4,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29
3,0.0,17.4775,14501,2011-01-10,1,100,699.0982,20110105,1,699.0982,...,"""Kostengünstiges Rad mit vielen Funktionen uns...",此自行车经济实惠，具有我们的高档车型所拥有的许多功能。相同的车灯、刚架以及我们驰名业界的快速...,R,2,1,Black,4,"Carretera: 650, negra, 62","Vélo de route 650 noir, 62",2010-12-29
4,0.0,84.9998,11003,2011-01-10,1,6,3399.9900,20110105,1,3399.9900,...,"""Top-Mountainbike für Profis. Mit innovativem ...",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,M,1,1,Silver,4,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29
5,0.0,89.4568,27645,2011-01-11,1,100,3578.2700,20110106,1,3578.2700,...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛...,R,2,1,Red,4,"Carretera: 150, roja, 44","Vélo de route 150 rouge, 44",2010-12-30
6,0.0,89.4568,16624,2011-01-11,1,6,3578.2700,20110106,1,3578.2700,...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛...,R,2,1,Red,4,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62",2010-12-30
7,0.0,84.3748,11005,2011-01-11,1,6,3374.9900,20110106,1,3374.9900,...,"""Top-Mountainbike für Profis. Mit innovativem ...",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,M,1,1,Black,4,"Montaña: 100, negra, 48","VTT 100 noir, 48",2010-12-30
8,0.0,84.9998,11011,2011-01-11,1,6,3399.9900,20110106,1,3399.9900,...,"""Top-Mountainbike für Profis. Mit innovativem ...",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地...,M,1,1,Silver,4,"Montaña: 100, plateada, 38","VTT 100 argent, 38",2010-12-30
9,0.0,89.4568,27621,2011-01-12,1,100,3578.2700,20110107,1,3578.2700,...,"""Dieses Fahrrad wird von Rennprofis gefahren. ...",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛...,R,2,1,Red,4,"Carretera: 150, roja, 48","Vélo de route 150 rouge, 48",2010-12-31


In [37]:
# truncate date to week in a new column

ds3 = ds2.datetrunc(dates={'ORDERDATEKEY_DATE':'week'})

ds3.preview()

,DISCOUNTAMOUNT,FREIGHT,REVISIONNUMBER,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,CARRIERTRACKINGNUMBER,...,PRODUCTSUBCATEGORYKEY,FINISHEDGOODSFLAG,LISTPRICE,ARABICDESCRIPTION,TURKISHDESCRIPTION,WEIGHTUNITMEASURECODE,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,ORDERDATEKEY_DATE,ORDERDATEKEY_DATE_WEEK
0,0.0,89.4568,1,21768,2011-01-10,1,19,3578.2700,20110105,None,...,2,1,3578.2700,إنها الدراجة التي يقودها الفائزون بسباقات الدر...,"""Bu bisiklet yarışçılar tarafından kullanılmak...",LB,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62",2010-12-29,2010-12-26
1,0.0,84.9998,1,28389,2011-01-10,1,39,3399.9900,20110105,None,...,1,1,3399.9900,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,"""En üst model rekabetçi dağ bisikleti. Perform...",LB,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29,2010-12-26
2,0.0,84.9998,1,25863,2011-01-10,1,100,3399.9900,20110105,None,...,1,1,3399.9900,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,"""En üst model rekabetçi dağ bisikleti. Perform...",LB,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29,2010-12-26
3,0.0,17.4775,1,14501,2011-01-10,1,100,699.0982,20110105,None,...,2,1,699.0982,إنها دراجة ذات سعر مناسب لإمكاناتها كما تتمتع ...,Değerine uygun olarak fiyatlandırılan bisiklet...,LB,"Carretera: 650, negra, 62","Vélo de route 650 noir, 62",2010-12-29,2010-12-26
4,0.0,84.9998,1,11003,2011-01-10,1,6,3399.9900,20110105,None,...,1,1,3399.9900,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,"""En üst model rekabetçi dağ bisikleti. Perform...",LB,"Montaña: 100, plateada, 44","VTT 100 argent, 44",2010-12-29,2010-12-26
5,0.0,89.4568,1,27645,2011-01-11,1,100,3578.2700,20110106,None,...,2,1,3578.2700,إنها الدراجة التي يقودها الفائزون بسباقات الدر...,"""Bu bisiklet yarışçılar tarafından kullanılmak...",LB,"Carretera: 150, roja, 44","Vélo de route 150 rouge, 44",2010-12-30,2010-12-26
6,0.0,89.4568,1,16624,2011-01-11,1,6,3578.2700,20110106,None,...,2,1,3578.2700,إنها الدراجة التي يقودها الفائزون بسباقات الدر...,"""Bu bisiklet yarışçılar tarafından kullanılmak...",LB,"Carretera: 150, roja, 62","Vélo de route 150 rouge, 62",2010-12-30,2010-12-26
7,0.0,84.3748,1,11005,2011-01-11,1,6,3374.9900,20110106,None,...,1,1,3374.9900,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,"""En üst model rekabetçi dağ bisikleti. Perform...",LB,"Montaña: 100, negra, 48","VTT 100 noir, 48",2010-12-30,2010-12-26
8,0.0,84.9998,1,11011,2011-01-11,1,6,3399.9900,20110106,None,...,1,1,3399.9900,دراجة سباقات مخصصة للقيادة في الجبال من أعلى ط...,"""En üst model rekabetçi dağ bisikleti. Perform...",LB,"Montaña: 100, plateada, 38","VTT 100 argent, 38",2010-12-30,2010-12-26
9,0.0,89.4568,1,27621,2011-01-12,1,100,3578.2700,20110107,None,...,2,1,3578.2700,إنها الدراجة التي يقودها الفائزون بسباقات الدر...,"""Bu bisiklet yarışçılar tarafından kullanılmak...",LB,"Carretera: 150, roja, 48","Vélo de route 150 rouge, 48",2010-12-31,2010-12-26


In [38]:
# aggregate sales to week
ds4 = ds3.aggregate(group_by=['ORDERDATEKEY_DATE_WEEK', 'PRODUCTKEY'],
  aggregations={'SALESAMOUNT': ['SUM', 'AVG'], 'TAXAMT': ['SUM'],'FREIGHT': ['SUM']}
)

ds4.preview()

,ORDERDATEKEY_DATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM
0,2010-12-26,310,7156.5400,3578.2700000000,572.5232,178.9136
1,2010-12-26,346,10199.9700,3399.9900000000,815.9976,254.9994
2,2010-12-26,336,699.0982,699.0982000000,55.9279,17.4775
3,2010-12-26,311,7156.5400,3578.2700000000,572.5232,178.9136
4,2010-12-26,351,3374.9900,3374.9900000000,269.9992,84.3748
5,2010-12-26,344,3399.9900,3399.9900000000,271.9992,84.9998
6,2010-12-26,330,699.0982,699.0982000000,55.9279,17.4775
7,2010-12-26,313,3578.2700,3578.2700000000,286.2616,89.4568
8,2010-12-26,314,7156.5400,3578.2700000000,572.5232,178.9136
9,2011-01-02,310,14313.0800,3578.2700000000,1145.0464,357.8272


In [44]:
print(ds4.sql)

CREATE OR REPLACE RASGO_OP_CEB93AB01CEF4C6AB55563AC5D165F4 AS SELECT
  t1.DUEDATEKEY, 
  t1.SALESORDERLINENUMBER, 
  t1.CUSTOMERKEY, 
  t1.DUEDATE, 
  t1.ORDERQUANTITY, 
  t1.CURRENCYKEY, 
  t1.UNITPRICE, 
  t1.SHIPDATEKEY, 
  t1.PROMOTIONKEY, 
  t1.REVISIONNUMBER, 
  t1.CUSTOMERPONUMBER, 
  t1.UNITPRICEDISCOUNTPCT, 
  t1.SHIPDATE, 
  t1.TOTALPRODUCTCOST, 
  t1.CARRIERTRACKINGNUMBER, 
  t1.ORDERDATE, 
  t1.SALESTERRITORYKEY, 
  t1.DISCOUNTAMOUNT, 
  t1.FREIGHT, 
  t1.EXTENDEDAMOUNT, 
  t1.PRODUCTKEY, 
  t1.SALESORDERNUMBER, 
  t1.ORDERDATEKEY, 
  t1.SALESAMOUNT, 
  t1.TAXAMT, 
  t1.PRODUCTSTANDARDCOST, t2.ENDDATE
    , t2.STARTDATE
    , t2.WEIGHT
    , t2.SIZE
    , t2.SIZEUNITMEASURECODE
    , t2.LARGEPHOTO
    , t2.PRODUCTALTERNATEKEY
    , t2.GERMANDESCRIPTION
    , t2.STYLE
    , t2.THAIDESCRIPTION
    , t2.ENGLISHDESCRIPTION
    , t2.HEBREWDESCRIPTION
    , t2.ENGLISHPRODUCTNAME
    , t2.FRENCHDESCRIPTION
    , t2.SAFETYSTOCKLEVEL
    , t2.SIZERANGE
    , t2.LISTPRICE
    , t2.ST

In [40]:
# add lags for forecast model

ds5 = ds4.lag(columns = ['SALESAMOUNT_SUM'],
    amounts = [1,2,3,4,26,52],
    order_by = ['ORDERDATEKEY_DATE_WEEK'],
    partition = ['PRODUCTKEY'])

ds5.preview()

,ORDERDATEKEY_DATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM,LAG_SALESAMOUNT_SUM_1,LAG_SALESAMOUNT_SUM_2,LAG_SALESAMOUNT_SUM_3,LAG_SALESAMOUNT_SUM_4,LAG_SALESAMOUNT_SUM_26,LAG_SALESAMOUNT_SUM_52
0,2013-01-06,475,69.9900,69.9900000000,5.5992,1.7498,None,None,None,None,None,None
1,2013-01-27,475,419.9400,69.9900000000,33.5952,10.4988,69.9900,None,None,None,None,None
2,2013-02-03,475,629.9100,69.9900000000,50.3928,15.7482,419.9400,69.9900,None,None,None,None
3,2013-02-10,475,209.9700,69.9900000000,16.7976,5.2494,629.9100,419.9400,69.9900,None,None,None
4,2013-02-17,475,279.9600,69.9900000000,22.3968,6.9992,209.9700,629.9100,419.9400,69.9900,None,None
5,2013-02-24,475,279.9600,69.9900000000,22.3968,6.9992,279.9600,209.9700,629.9100,419.9400,None,None
6,2013-03-03,475,699.9000,69.9900000000,55.9920,17.4980,279.9600,279.9600,209.9700,629.9100,None,None
7,2013-03-10,475,419.9400,69.9900000000,33.5952,10.4988,699.9000,279.9600,279.9600,209.9700,None,None
8,2013-03-17,475,629.9100,69.9900000000,50.3928,15.7482,419.9400,699.9000,279.9600,279.9600,None,None
9,2013-03-24,475,629.9100,69.9900000000,50.3928,15.7482,629.9100,419.9400,699.9000,279.9600,None,None


In [42]:
# apply 80/20 train-test split

modeling_ds = ds5.train_test_split(order_by = ['ORDERDATEKEY_DATE_WEEK'],train_percent = .8)

modeling_ds.preview()

,ORDERDATEKEY_DATE_WEEK,PRODUCTKEY,SALESAMOUNT_SUM,SALESAMOUNT_AVG,TAXAMT_SUM,FREIGHT_SUM,LAG_SALESAMOUNT_SUM_1,LAG_SALESAMOUNT_SUM_2,LAG_SALESAMOUNT_SUM_3,LAG_SALESAMOUNT_SUM_4,LAG_SALESAMOUNT_SUM_26,LAG_SALESAMOUNT_SUM_52,TT_SPLIT
0,2010-12-26,344,3399.9900,3399.9900000000,271.9992,84.9998,None,None,None,None,None,None,TRAIN
1,2010-12-26,346,10199.9700,3399.9900000000,815.9976,254.9994,None,None,None,None,None,None,TRAIN
2,2010-12-26,310,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,TRAIN
3,2010-12-26,313,3578.2700,3578.2700000000,286.2616,89.4568,None,None,None,None,None,None,TRAIN
4,2010-12-26,336,699.0982,699.0982000000,55.9279,17.4775,None,None,None,None,None,None,TRAIN
5,2010-12-26,312,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,TRAIN
6,2010-12-26,330,699.0982,699.0982000000,55.9279,17.4775,None,None,None,None,None,None,TRAIN
7,2010-12-26,314,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,TRAIN
8,2010-12-26,351,3374.9900,3374.9900000000,269.9992,84.3748,None,None,None,None,None,None,TRAIN
9,2010-12-26,311,7156.5400,3578.2700000000,572.5232,178.9136,None,None,None,None,None,None,TRAIN


In [43]:
# publish the new data set back to Rasgo
created = rasgo.publish.dataset(dataset=modeling_ds, name='WeeklySalesModel', description='Product Sales by Week')
print(created)

In [ ]:
# follow this URL to see your new dataset in the Rasgo UI

print('https://app.rasgoml.com/datasets/{}'.format(created.id))

<h1>COMING SOON</h1>
<h2>Example 2: Analyze Customer Behavior</h2>